In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Managing investment strategies

Demonstration of how to compare how strategies are performing across all of our client's holdings, rather than just looking at a single portfolio in isolation.

Attributes
----------
instruments
transactions
portfolio groups
properties
set holdings
data types
aggregation
"""

toggle_code("Toggle Docstring")

## The Challenge

We are an asset manager who has just taken on a new client. The client has given us a number of mandates on how to manage their assets and we have assigned each of these to a different portfolio manager.

Even though there are a number of different mandates, some of the strategies that we are using are common across mandates. We really need to be able to see how each strategy is performing across all of our client's holdings, rather than just looking at a single portfolio in isolation.

## The Solution

Using LUSID we have the ability to add a strategy label to each transaction that we make. We can then aggregate across these labels to get a holistic view into how a strategy is performing. We can do this by:

1. Creating a portfolio for each mandate that our client has given us and group them in a portfolio group
2. Loading our instrument universe 
3. Adding our take on balances for each portfolio which have been provided to us by a transition manager
4. Creating a strategy tag that we can use to tag our transactions with the strategy used to make the trade
5. Adding the first trades that have been made by our portfolio managers inlcuding the strategy that they were following when they made them
6. Adding market data quotes so that we can aggregate across our client's portfolios
7. Aggregate across all of our client's portfolios to see the overall position
8. Aggregate across all of our client's portfolios by strategy to our position per strategy

First things first though, we need to initialise our LUSID environment. 

*To initialise our LUSID environment run the cell below*

In [2]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import os

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('LUSID API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.6.6625.0


We now have an empty LUSID environment initialised

![Scopes](img/strategies/strategies-lusid.gif)

### 1) Creating a portfolio for each mandate and grouping them in a portfolio group

In LUSID we can create separate environments for each of our clients using scopes. A scope is a container for LUSID objects and can be thought of as a separate identity namespace. Using LUSID's entitlements engine we can ensure that only the right people can access a given scope. Furthermore we can ensure that only the right people can access the right objects inside a given scope. LUSID allows for entitlements down to the most granular level. 

For this example we will create a scope for our new client. Let us assume that our new client is a pension fund for a large high street retailer which we will call 'high_street_pension_fund'.

*Run the cell below to initialise our scope. Note that the scope will have a unique 4 character code appended to it to make the name more unique.*

In [3]:
# Fetch our scopes
scope_id = import_data.create_scope_id()
client_scope_code = 'client-high_street_pension_fund-{}'.format(scope_id)
# Pretty print our scope
prettyprint.heading('New Client Scope Code', client_scope_code)

New Client Scope Code: client-high_street_pension_fund-396e-1e2f-4280-08


We now have a scope for our client inside our LUSID environment. 

![Scopes](img/strategies/strategies-scope.gif)

Now that we have our scope we can create our portfolios.

The first thing that we need to do here is give each portfolio a unique name. In LUSID these are reffered to as 'codes'. Almost every object in LUSID can be refered to by the combination of the scope that it is contained in and its code. The code must be unique to the scope.

Our pension fund has five mandates and thus five portfolios. 

In LUSID we can use portfolio groups to group the different portfolios of each of our clients together. This allows us to easily keep track of and work with the holdings of each of our clients.

*Run the cell below to import the name for our portfolio group and its corresponding portfolios.*

In [4]:
client_portfolios = pd.read_csv('data/strategy-portfolios.csv')
client_portfolios.head()

,portfolio_group_name,portfolio_name
0,high_street_pension_fund-portfolios,mandate-balanced
1,high_street_pension_fund-portfolios,mandate-energy
2,high_street_pension_fund-portfolios,mandate-fixedincome
3,high_street_pension_fund-portfolios,mandate-international
4,high_street_pension_fund-portfolios,mandate-usgovt


Note that in LUSID we can only create one portfolio at a time. 
        
We will thus create each of the portfolios for our client one after the other and then we will create the portfolio group that binds them together.
        
We on-boarded this clients 5 days ago. When we create the portfolios we can set the creation date using the 'created' argument. This will allow us to backdate holdings and transactions.

You can read more about creating a portfolio in the LUSID API documentation: [LUSID API Docs: Creating a Portfolio](https://docs.lusid.com/#operation/CreatePortfolio)

You can also read more about creating a portfolio group in the LUSID API documentation:

[LUSID API Docs: Creating a Portfolio Group](https://docs.lusid.com/#operation/CreatePortfolioGroup)

*Run the cell below to create our portfolios and group them together*

In [5]:
# Set the creation date of our portfolio
created_date = datetime.now(pytz.UTC) - timedelta(days=5)

# Initialise a list to hold the ids of our portfolios for use in creating our portfolio group
portfolio_resourceids = []

# Loop over our portfolios
for portfolio_code in client_portfolios['portfolio_name'].unique():
    
    # Create the request to add our portfolio
    portfolio_request = models.CreateTransactionPortfolioRequest(
        display_name=portfolio_code,
        code=portfolio_code,
        base_currency='GBP',
        description=portfolio_code,
        created=created_date)
    
    # Call LUSID to create our portfolio
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
        scope=client_scope_code,
        create_transaction_portfolio_request=portfolio_request)

    # Print the response from LUSID using pretty formatting
    prettyprint.portfolio_response(response)

    # Add the portfolio to our list of portfolios for the portfolio group
    portfolio_resourceids.append(
        models.ResourceId(
            scope=client_scope_code, 
            code=portfolio_code)
    )

client_group_code = client_portfolios['portfolio_group_name'].unique()[0]
    
# Create our portfolio group request
portfolio_group_request = models.CreatePortfolioGroupRequest(
    code=client_group_code,
    display_name=client_group_code,
    values=portfolio_resourceids,
    created=created_date,
    description='Grouping all of {}'.format(client_group_code))

# Call LUSID to create our portfolio group
response = api_factory.build(lusid.api.PortfolioGroupsApi).create_portfolio_group(
    scope=client_scope_code,
    create_portfolio_group_request=portfolio_group_request)

# Print the response from LUSID using pretty formatting
prettyprint.portfolio_group_response(response, 'created')

Portfolio Created
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: mandate-balanced
Portfolio Effective From: 2021-03-18 15:55:59.365049+00:00
Portfolio Created On: 2021-03-23 15:55:59.694293+00:00

Portfolio Created
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: mandate-energy
Portfolio Effective From: 2021-03-18 15:55:59.365049+00:00
Portfolio Created On: 2021-03-23 15:56:00.416269+00:00

Portfolio Created
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: mandate-fixedincome
Portfolio Effective From: 2021-03-18 15:55:59.365049+00:00
Portfolio Created On: 2021-03-23 15:56:00.997271+00:00

Portfolio Created
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: mandate-international
Portfolio Effective From: 2021-03-18 15:55:59.365049+00:00
Portfolio Created On: 2021-03-23 15:56:01.370215+00:00

Portfolio Created
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: mandate-usgovt
Portfolio Effective From: 2021-03-18 15:55:59.

Our scope for our new client is now populated with our portfolios which are grouped together in a portfolio group. All of our portfolios are currently empty.

![Scopes](img/strategies/strategies-portfolios.gif)

### 2) Adding our instrument universe

Before we can load our take on balances we first need to add securities to our instrument master in LUSID. 
        
The instrument master contains the details of all of the securities that we hold or trade in. Any time that we conduct a trade or add a holding of a security we refer to it in the instrument master. 

Below we can import our universe of instruments which all of our holdings are made up of. Using LUSID this universe can be pre-populated in advance with your entire instrument universe so that we don't have to keep adding new instruments when we make a trade or update a holding. 

*Run the cell below to import our instrument universe*

In [6]:
instrument_universe = pd.read_csv('data/instruments.csv')
instrument_universe.head(n=15)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,nan
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,nan
2,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,LN,united_kingdom,BP/,equity,common_stock,nan
3,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,LN,united_kingdom,BUR,equity,common_stock,nan
4,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,LN,united_kingdom,EKF,equity,common_stock,nan
5,Glencore_LondonStockEx_GLEN,imd_34534555,GBP,JE00B4T3BW64,BBG001MM1KV4,LN,united_kingdom,GLEN,equity,common_stock,nan
6,JustEat_LondonStockEx_JE,imd_35436366,GBP,GB00BKX5CN86,BBG0065YWM39,LN,united_kingdom,JE/,equity,common_stock,nan
7,Kingfisher_LondonStockEx_KGF,imd_34535552,GBP,GB0033195214,BBG000BKH1W6,LN,united_kingdom,KGF,equity,common_stock,nan
8,MicroFocus_LondonStockEx_MCRO,imd_34567338,GBP,GB00BD8YWM01,BBG000G4KKD2,LN,united_kingdom,MCRO,equity,common_stock,nan
9,RELXGroup_LondonStockEx_REL,imd_43532542,GBP,GB00B2B0DG97,BBG000D03XD4,LN,united_kingdom,REL,equity,common_stock,nan


Now that we have our instruments we can add them to LUSID.

To add securities to the instrument master in LUSID we use an upsert method. This method will insert a new record if it does not exist and update the existing record if it does exist. This allows us to add instruments more simply than have to check if an instrument exists before we insert it. 

We can upsert our instruments individually or in a batch. You can read more about upserting instruments
in the LUSID API Documentation: [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments)

*Run the cell below to upsert our instrument universe*

In [7]:
# Initialise your batch upsert request
batch_upsert_request = {}

# Using your instrument universe create your batch request
for index, instrument in instrument_universe.iterrows():

    # Specify the columns of your identifiers
    identifier_columns = [
            ('isin', 'Isin'), 
            ('figi', 'Figi'), 
            ('ticker', 'Ticker'),
            ('client_internal', 'ClientInternal')
    ]
    
    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier[1]] = models.InstrumentIdValue(
            value=instrument[identifier[0]])
    
    # Build your request and add it to the dictionary
    batch_upsert_request[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers)
    
# Call LUSID to upsert your instrument defintions
instrument_response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(request_body=batch_upsert_request)

# Pretty print the response
prettyprint.instrument_response(instrument_response)

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,MicroFocus_LondonStockEx_MCRO,imd_34567338,LUID_AEHHP893
1,UKGiltTreasury_4.5_2034,imd_34534534,LUID_6CHCCKIV
2,UKGiltTreasury_2.0_2025,imd_34534536,LUID_G0Y0H4TM
3,Amazon_Nasdaq_AMZN,imd_34634534,LUID_PTYEYW95
4,USTreasury_2.00_2021,imd_34535347,LUID_TNXH4ZLD
5,USTreasury_6.875_2025,imd_34534539,LUID_OCHBK89B
6,Glencore_LondonStockEx_GLEN,imd_34534555,LUID_5T7DZTUG
7,JustEat_LondonStockEx_JE,imd_35436366,LUID_76A8ZKWK
8,Kingfisher_LondonStockEx_KGF,imd_34535552,LUID_62ZKJUTN
9,UKGiltTreasury_3.75_2021,imd_34643653,LUID_85PTLJWK


Our LUSID environment now has an instrument master containing all of our instruments that we will be trading.

![Scopes](img/strategies/strategies-instrumentmaster.gif)

### 3) Adding our take on balances for each portfolio which have been provided to us by a transition manager

We have been advised by a transition manager on the holdings that we will be taking on for our new client. They have provided these initial holdings to us in a CSV.

Let us load our take on balances. 

*Run the cell below to import our initial holdings information*

In [8]:
client_holdings = pd.read_csv('data/strategy-holdings.csv')
client_holdings.head(n=20)

,portfolio_group_name,portfolio_name,instrument_name,quantity,price,currency,figi
0,high_street_pension_fund-portfolios,mandate-balanced,Kingfisher_LondonStockEx_KGF,1362038,2.28,GBP,BBG000BKH1W6
1,high_street_pension_fund-portfolios,mandate-balanced,JustEat_LondonStockEx_JE,834553,5.46,GBP,BBG0065YWM39
2,high_street_pension_fund-portfolios,mandate-balanced,RELXGroup_LondonStockEx_REL,494343,15.98,GBP,BBG000D03XD4
3,high_street_pension_fund-portfolios,mandate-balanced,UKGiltTreasury_4.5_2034,77481,140.57,GBP,BBG0000D14P3
4,high_street_pension_fund-portfolios,mandate-balanced,GBP_Cash,952000,1.00,GBP,NaN
5,high_street_pension_fund-portfolios,mandate-energy,Glencore_LondonStockEx_GLEN,905141,2.76,GBP,BBG001MM1KV4
6,high_street_pension_fund-portfolios,mandate-energy,BP_LondonStockEx_BP,1713922,5.11,GBP,BBG000C05BD1
7,high_street_pension_fund-portfolios,mandate-energy,GBP_Cash,2200000,1.00,GBP,NaN
8,high_street_pension_fund-portfolios,mandate-fixedincome,UKGiltTreasury_3.5_2045,266169,134.43,GBP,BBG006N6HZM7
9,high_street_pension_fund-portfolios,mandate-fixedincome,UKGiltTreasury_2.0_2025,405589,106.64,GBP,BBG0088JSC32


To set our holdings in LUSID we use the set holdings method. This method takes a list of adjustment requests. Using these requests LUSID will determine what transactions are required to update our portfolios so that they have the desired holdings.

We will make our holdings effective as of five days ago as this was when we on-boarded the client. 

Just like when we created our portfolios we can only set the holdings one portfolio at a time. 

You can read more about setting the holdings on a portfolio in the LUSID API Documentation: [LUSID API Docs: Setting the Holdings on a Portfolio](https://docs.lusid.com/#operation/SetHoldings)

*Run the cell below to set our take on balances*

In [9]:
holdings_effective_date = (datetime.now(pytz.UTC) - timedelta(days=5))

# Iterate over our portfolios
for portfolio_name in client_holdings['portfolio_name'].unique():
    
    # Initialise our list to hold the holding adjustments
    holding_adjustments = []

    # Iterate over the holdings in each portfolio
    for index, holding in client_holdings.loc[client_holdings['portfolio_name'] == portfolio_name].iterrows():
        
        # Get the currency of the holding
        currency = holding['currency']
        
        # Identify and handle cash instruments
        if 'Cash' in holding['instrument_name']:
            identifier_key = 'Instrument/default/Currency'
            identifier = currency
        # Else treat it as a normal instrument and use Figi
        else:
            identifier_key = 'Instrument/default/Figi'
            identifier = holding['figi']

        # Create our adjust holdings request using our instrument universe to get the LUID identifier for the instrument
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: identifier},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['quantity'] * holding['price'],
                            currency=currency),
                        portfolio_cost=holding['quantity'] * holding['price'],
                        price=holding['price'])
                ])
        )
    
    # Call LUSID to set our holdings
    set_holdings_response = api_factory.build(lusid.api.TransactionPortfoliosApi).set_holdings(
        scope=client_scope_code,
        code=portfolio_name,
        effective_at=holdings_effective_date,
        adjust_holding_request=holding_adjustments)
    
    # Print the response from LUSID using pretty formatting
    prettyprint.set_holdings_response(
        set_holdings_response, 
        client_scope_code, 
        portfolio_name)

print ('Holdings have been set for all Portfolios')

Holdings Successfully Set for Portfolio
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: mandate-balanced
Holdings Effective From: 2021-03-18 15:55:59.365049+00:00
Holdings Created On: 2021-03-23 15:56:03.702628+00:00

Holdings Successfully Set for Portfolio
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: mandate-energy
Holdings Effective From: 2021-03-18 15:55:59.365049+00:00
Holdings Created On: 2021-03-23 15:56:04.465537+00:00

Holdings Successfully Set for Portfolio
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: mandate-fixedincome
Holdings Effective From: 2021-03-18 15:55:59.365049+00:00
Holdings Created On: 2021-03-23 15:56:05.081831+00:00

Holdings Successfully Set for Portfolio
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: mandate-international
Holdings Effective From: 2021-03-18 15:55:59.365049+00:00
Holdings Created On: 2021-03-23 15:56:05.764915+00:00

Holdings Successfully Set for Portfolio
Scope: client-high_str

Our portfolios are no longer empty and now contain the initial holdings for our new client.

![Scopes](img/strategies/strategies-holdings.gif)

### 4) Creating a strategy tag that we can use to tag our transactions with the strategy used to make the trade

We want to be able to track what strategies our portfolio managers were following when they made an investment decision. To do this in LUSID we can use properties. We can define a property and attach it to almost any object in LUSID. 

In this case we can create a 'strategy' property that we can attach to our transactions. This property will take the form of a string which will be the name of the strategy that the portfolio manager followed. Examples of strategies in this case are 'quantitativeSignal', 'fundamentalAnalysis' etc.

To ensure that the there are no issues with things such as mispelled strategies we will create a custom data type to use with our property which restricts the available options.

You can read more about defining new properties in the LUSID API documentation: [LUSID API Docs: Creating a New Data Type](https://www.lusid.com/docs/api/#operation/CreateDataType)

*Run the cell below to create our custom data type*

In [10]:
create_request = lusid.models.CreateDataTypeRequest(
    scope=client_scope_code,
    code="StrategyCodes",
    type_value_range="Closed",
    display_name="Available Investment Strategy Tags",
    description="This data type contains the available strategies which can be used",
    value_type="String",
    acceptable_values=[
        "quantitativeSignal",
        "incomeRequirements",
        "internationalExposure",
        "fundamentalAnalysis",
        "newsSentiment"
    ])


response = api_factory.build(lusid.api.DataTypesApi).create_data_type(
    create_data_type_request=create_request)

print("Data Type Created")
prettyprint.heading("Scope", response.id.scope)
prettyprint.heading("Code", response.id.code)

Data Type Created
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: StrategyCodes


We can now create a property definition using our custom data type. 

You can read more about defining new properties in the LUSID API documentation: [LUSID API Docs: Creating a New Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition)

*Run the cell below to create our strategy property*

In [11]:
# Create our request to define a new property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Transaction',
    scope=client_scope_code,
    code='strategy',
    value_required=False,
    display_name='strategy',
    data_type_id=models.ResourceId(scope=client_scope_code, code='StrategyCodes'))

# Call LUSID to create our new property
property_response = api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
    create_property_definition_request=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
strategy_property_key = property_response.key

# Pretty print our key
prettyprint.heading('Strategy Property Key', strategy_property_key)

Strategy Property Key: Transaction/client-high_street_pension_fund-396e-1e2f-4280-08/strategy


We now have a strategy property that we can set on each of our transactions which is referenced by our strategy property key.

![Scopes](img/strategies/strategies-strategypropertykey.gif)

### 5) Adding the first trades that have been made by our portfolio managers including the strategy that they were following when they made them

Now that we have our transaction property defined we can load our transactions. We will load these from a CSV file. In practice these are likely to come from our order management system. You can see from looking at the transaction dates that these transactions have all occured over the last 4 days.

*Run the cell below to import our transactions*

In [12]:
client_transactions = import_data.fetch_client_transactions(
    'strategy-transactions.csv', 
    days_back=4)

client_transactions.head(n=10)

,transaction_id,type,portfolio,instrument_name,instrument_uid,transaction_date,settlement_date,units,transaction_price,transaction_currency,total_cost,strategy,description
0,tid_329432525234324,Sell,mandate-balanced,Kingfisher_LondonStockEx_KGF,BBG000BKH1W6,2021-03-22T15:30:43.218858+00:00,2021-03-24T15:30:43.218858+00:00,325000,2.35,GBP,762125.00,quantitativeSignal,Equity Sale
1,tid_325452342424500,Buy,mandate-balanced,UKGiltTreasury_4.5_2034,BBG0000D14P3,2021-03-21T13:41:22.617131+00:00,2021-03-23T13:41:22.617131+00:00,10501,140.57,GBP,1476146.57,incomeRequirements,Equity Purchase
2,tid_234295929052090,Buy,mandate-fixedincome,UKGiltTreasury_3.75_2021,BBG001KKJLR4,2021-03-19T15:48:55.359595+00:00,2021-03-21T15:48:55.359595+00:00,24000,109.13,GBP,2619024.00,incomeRequirements,Equity Purchase
3,tid_234942982496001,Sell,mandate-fixedincome,USTreasury_2.00_2021,BBG00FN3B5K8,2021-03-21T08:14:29.724875+00:00,2021-03-23T08:14:29.724875+00:00,57000,97.80,USD,5574600.00,internationalExposure,Equity Sale
4,tid_239491298599900,Sell,mandate-energy,Glencore_LondonStockEx_GLEN,BBG001MM1KV4,2021-03-20T10:44:56.462239+00:00,2021-03-22T10:44:56.462239+00:00,120000,2.60,GBP,312000.00,fundamentalAnalysis,Equity Sale
5,tid_121309590059995,Buy,mandate-energy,BP_LondonStockEx_BP,BBG000C05BD1,2021-03-20T16:38:02.661344+00:00,2021-03-22T16:38:02.661344+00:00,50000,5.29,GBP,264350.00,fundamentalAnalysis,Equity Purchase
6,tid_124319009950924,Sell,mandate-usgovt,USTreasury_2.00_2021,BBG00FN3B5K8,2021-03-20T13:28:44.499930+00:00,2021-03-22T13:28:44.499930+00:00,28000,98.10,USD,2746800.00,incomeRequirements,Equity Sale
7,tid_212390582999593,Buy,mandate-international,Apple_Nasdaq_AAPL,BBG000B9XVV8,2021-03-22T09:40:11.227813+00:00,2021-03-24T09:40:11.227813+00:00,15000,168.49,GBP,2527350.00,newsSentiment,Equity Purchase


Now that we have our transactions we can upsert them into LUSID. We can only upsert transactions for one portfolio at time. Therefore we will upsert our transactions in batches. Each batch contains all the transactions for a given portfolio. 

You can read more about upserting transactions in the LUSID API documentation: [LUSID API Docs: Upserting Transactions](https://docs.lusid.com/#operation/UpsertTransactions)

*Run the cell below to upsert our transactions*

In [13]:
# Iterate over our portfolios
for portfolio_name in client_transactions['portfolio'].unique():
    # Initialise a list to hold our transactions for each portfolio
    batch_transaction_requests = []
    # Iterate over the transactions for each portfolio
    for index, transaction in client_transactions.loc[
        client_transactions['portfolio'] == portfolio_name].iterrows():
        # Append the transaction to our request, note the use of the 'properties' argument
        batch_transaction_requests.append(
            models.TransactionRequest(
                transaction_id=transaction['transaction_id'],
                type=transaction['type'],
                instrument_identifiers={
                    'Instrument/default/Figi': transaction['instrument_uid']},
                transaction_date=transaction['transaction_date'],
                settlement_date=transaction['settlement_date'],
                units=transaction['units'],
                transaction_price=models.TransactionPrice(
                  price=transaction['transaction_price'],
                  type='Price'),
                total_consideration=models.CurrencyAndAmount(
                  amount=transaction['total_cost'],
                  currency=transaction['transaction_currency']),
                 source='Client',
                 transaction_currency=transaction['transaction_currency'],
                 properties={
                     strategy_property_key: models.PerpetualProperty(
                         key=strategy_property_key,
                         value=models.PropertyValue(label_value=transaction['strategy'])
                     )
                 }
              ))
            
    # Call LUSID to upsert our transactions
    transaction_response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
        scope=client_scope_code,
        code=portfolio_name,
        transaction_request=batch_transaction_requests)
    
    # Print the response from LUSID using pretty formatting 
    prettyprint.transactions_response(
        transaction_response,
        client_scope_code,
        portfolio_name)

Transactions Successfully Upserted into Portfolio
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: mandate-balanced
Transactions Effective From: 2021-03-22 15:30:43.218858+00:00
Transactions Created On: 2021-03-23 15:56:07.822944+00:00

Transactions Successfully Upserted into Portfolio
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: mandate-fixedincome
Transactions Effective From: 2021-03-21 08:14:29.724875+00:00
Transactions Created On: 2021-03-23 15:56:08.565683+00:00

Transactions Successfully Upserted into Portfolio
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: mandate-energy
Transactions Effective From: 2021-03-20 16:38:02.661344+00:00
Transactions Created On: 2021-03-23 15:56:09.381722+00:00

Transactions Successfully Upserted into Portfolio
Scope: client-high_street_pension_fund-396e-1e2f-4280-08
Code: mandate-usgovt
Transactions Effective From: 2021-03-20 13:28:44.499930+00:00
Transactions Created On: 2021-03-23 15:56:09.948934+00:0

Now each of our portfolios contains a list of transactions, the first are 'StockIn' transactions generated by LUSID to set our holdings, the second set are the transactions from our portfolio managers which are tagged with a strategy.

![Scopes](img/strategies/strategies-transactions.gif)

### 6) Adding market data quotes so that we can aggregate across our client's portfolios

Now that we have populated our portfolios we want to aggregate across them to see both our client's overall position as well as our exposure as the result of each strategy that our portfolio managers are using. 

To this we first need to add some market data to the LUSID quote store.

You can read more about quotes in the LUSID API documentation: [LUSID API Docs: Quotes](https://support.lusid.com/what-is-a-quote)

*Run the cell below to create add our quotes*

In [14]:
# Set our analytics effective date to be from now
quotes_effective_date = datetime.now(pytz.UTC)
instrument_quotes = {}

# Create dummy prices of 1 for all instruments except cash which we will ignore
for index, instrument in instrument_universe.iterrows():
    
    if 'Cash' in instrument['instrument_name']:
        continue
    
    luid = api_factory.build(lusid.api.InstrumentsApi).get_instrument(
        identifier_type='Figi',
        identifier=instrument['figi']).lusid_instrument_id
    
    instrument_quotes[luid] = models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider='DataScope',
                    instrument_id=luid,
                    instrument_id_type='LusidInstrumentId',
                    quote_type='Price',
                    field='Mid'),
                effective_at=quotes_effective_date
            ),
            metric_value=models.MetricValue(
                value=1,
                unit="GBP"),
            lineage='InternalSystem')

# Upsert the quotes into LUSID
response = api_factory.build(lusid.api.QuotesApi).upsert_quotes(
    scope=client_scope_code,
    request_body=instrument_quotes)

# Pretty print the response from LUSID
prettyprint.upsert_quotes_response(response)

,_lineage,_cut_label,_uploaded_by,_as_at,_scale_factor,discriminator,_provider,_price_source,_instrument_id,_instrument_id_type,_quote_type,_field,_value,_unit,status
0,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:56:12.234767+00:00,None,None,DataScope,None,LUID_PTYEYW95,LusidInstrumentId,Price,Mid,1.00,GBP,Success
1,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:56:12.234767+00:00,None,None,DataScope,None,LUID_BP36C9G6,LusidInstrumentId,Price,Mid,1.00,GBP,Success
2,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:56:12.234767+00:00,None,None,DataScope,None,LUID_QFAFBL1W,LusidInstrumentId,Price,Mid,1.00,GBP,Success
3,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:56:12.234767+00:00,None,None,DataScope,None,LUID_TS2R4MAS,LusidInstrumentId,Price,Mid,1.00,GBP,Success
4,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:56:12.234767+00:00,None,None,DataScope,None,LUID_EK70ZAUS,LusidInstrumentId,Price,Mid,1.00,GBP,Success
5,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:56:12.234767+00:00,None,None,DataScope,None,LUID_5T7DZTUG,LusidInstrumentId,Price,Mid,1.00,GBP,Success
6,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:56:12.234767+00:00,None,None,DataScope,None,LUID_76A8ZKWK,LusidInstrumentId,Price,Mid,1.00,GBP,Success
7,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:56:12.234767+00:00,None,None,DataScope,None,LUID_62ZKJUTN,LusidInstrumentId,Price,Mid,1.00,GBP,Success
8,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:56:12.234767+00:00,None,None,DataScope,None,LUID_AEHHP893,LusidInstrumentId,Price,Mid,1.00,GBP,Success
9,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:56:12.234767+00:00,None,None,DataScope,None,LUID_D7LTB603,LusidInstrumentId,Price,Mid,1.00,GBP,Success


We now have market data in our new client's scope that we can use for our aggregation requests.

![Scopes](img/strategies/strategies-quotestore.gif)

### 7) Aggregate across all of our client's portfolios to see the overall position

The first thing we want to do is aggregate across our client's entire group of portfolios to see their overall position. We can do this by aggregating the holdings by the portfolio group. We will sum the units and the cost of each instrument across the group.

You can read more about creating by portfolio group in the LUSID API documentation: [LUSID API Docs: Aggregate by Portfolio Group](https://docs.lusid.com/#operation/GetAggregationByGroup)

*Run the cell below to aggregate across all of our client's portfolios*

In [15]:
recipe_code = "market_value"
recipe_scope = "strategies"

# Create a recipe to perform a valuation
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

configuration_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
               key='Equity.LusidInstrumentId.*',
               supplier='DataScope',
               data_scope=client_scope_code,
               quote_type='Price',
               field='Mid')
        ],
        suppliers=models.MarketContextSuppliers(
            commodity='DataScope',
            credit='DataScope',
            equity='DataScope',
            fx='DataScope',
            rates='DataScope'),
        options=models.MarketOptions(
            default_supplier='DataScope',
            default_instrument_code_type='LusidInstrumentId',
            default_scope=client_scope_code)
        )
    )

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request = models.UpsertRecipeRequest(
    configuration_recipe = configuration_recipe
    )
)

In [16]:
# Create the valuation request
valuation_request = models.ValuationRequest(
    recipe_id= models.ResourceId(scope=recipe_scope, code=recipe_code),
    metrics=[
        models.AggregateSpec(key='Instrument/default/LusidInstrumentId',
        op='Value'),
        models.AggregateSpec(key='Instrument/default/Name',
        op='Value'),
        models.AggregateSpec(key='Holding/default/Units',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum')
    ],
    group_by=["Instrument/default/LusidInstrumentId"],
    portfolio_entity_ids=[models.PortfolioEntityId(scope=client_scope_code,
                                                   code=client_group_code,
                                                   portfolio_entity_type="GroupPortfolio")],
    valuation_schedule=models.ValuationSchedule(effective_at=datetime.now(pytz.UTC).isoformat())
)

# Perform a valuation
valuation = api_factory.build(lusid.api.AggregationApi).get_valuation(
    valuation_request=valuation_request)

pd.DataFrame(valuation.data)

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Sum(Holding/default/Units),Sum(Holding/default/Cost)
0,LUID_5T7DZTUG,Glencore_LondonStockEx_GLEN,785141.00,2168559.44
1,LUID_QFAFBL1W,BP_LondonStockEx_BP,1763922.00,9029347.11
2,CCY_GBP,GBP,789254.43,789254.43
3,LUID_62ZKJUTN,Kingfisher_LondonStockEx_KGF,1037038.00,2360298.49
4,LUID_76A8ZKWK,JustEat_LondonStockEx_JE,834553.00,4559997.59
5,LUID_D7LTB603,RELXGroup_LondonStockEx_REL,494343.00,7899601.14
6,LUID_6CHCCKIV,UKGiltTreasury_4.5_2034,87982.00,12367805.70
7,LUID_D8STZ7MA,UKGiltTreasury_3.5_2045,266169.00,35781897.18
8,LUID_G0Y0H4TM,UKGiltTreasury_2.0_2025,405589.00,43250794.19
9,LUID_85PTLJWK,UKGiltTreasury_3.75_2021,198800.00,21519448.80


Now we have a view of our client's holdings across their entire group of portfolios.

![Scopes](img/strategies/strategies-aggregatedportfoliogroup.gif)

### 8) Aggregate across all of our client's portfolios by strategy to our position per strategy

Now that we have a view of our client's overall position we want to get a sense as to what strategies our portfolio manager's have been using to make their trades and how our position looks by strategy.

To do this we will make us of Derived Portfolios. 

A derived portfolio is a portfolio that is derived from a parent portfolio. It inherits all of its transactions from the parent portfolio. We can make use of derived portfolios to apply sub-holding keys to our holdings. In this case we will use our transaction strategy property as our sub-holding key.

We will create a new scope to hold our derived portfolios. This will allow us to give them the same name as the parent portfolio.

*Run the cell below to create our new scope*

In [17]:
# Fetch our scopes
scope_id = import_data.create_scope_id()
client_strategy_scope_code = 'client-strategy-high_street_pension_fund-{}'.format(scope_id)
# Pretty print our scope
prettyprint.heading('New Client Strategy Scope Code', client_strategy_scope_code)

New Client Strategy Scope Code: client-strategy-high_street_pension_fund-396e-1e38-9095-26


Now that we have our new scope we can create our derived portfolios. We will create one for each of the corresponding parent portfolios. We will also group our derived portfolios together so that we can aggregate across the group.

Note that we use the same creation date for the derived portfolios as we did for the parent portfolio. 

You can read more about creating a derived portfolio in the LUSID API documentation: [LUSID API Docs: Creating a Derived Portfolio](https://docs.lusid.com/#operation/CreateDerivedPortfolio)

*Run the cell below to create and group our derived portfolios*

In [18]:
# Initialise a list to hold the ids of our portfolios for use in creating our portfolio group
portfolio_resourceids = []

# Loop over our portfolios
for portfolio_code in client_portfolios['portfolio_name'].unique():
    
    # Create the request to add our portfolio
    portfolio_request = models.CreateDerivedTransactionPortfolioRequest(
        display_name=portfolio_code,
        code=portfolio_code,
        parent_portfolio_id=models.ResourceId(
            scope=client_scope_code,
            code=portfolio_code),
        description=portfolio_code,
        created=created_date,
        sub_holding_keys=[strategy_property_key])
    
    # Call LUSID to create our portfolio
    portfolio_response = api_factory.build(lusid.api.DerivedTransactionPortfoliosApi).create_derived_portfolio(
        scope=client_strategy_scope_code,
        create_derived_transaction_portfolio_request=portfolio_request)
    
    # Print the response from LUSID using pretty formatting
    prettyprint.portfolio_response(portfolio_response)

    # Add the portfolio to our list of portfolios for the portfolio group
    portfolio_resourceids.append(
        models.ResourceId(
            scope=client_strategy_scope_code, 
            code=portfolio_code))

# Create our portfolio group request
portfolio_group_request = models.CreatePortfolioGroupRequest(
    code=client_group_code,
    display_name=client_group_code,
    values=portfolio_resourceids,
    created=created_date,
    description='Grouping all of {}'.format(client_group_code))

# Call LUSID to create our portfolio group
response = api_factory.build(lusid.api.PortfolioGroupsApi).create_portfolio_group(
    scope=client_strategy_scope_code,
    create_portfolio_group_request=portfolio_group_request)

# Print the response from LUSID using pretty formatting
prettyprint.portfolio_group_response(response, 'created')

Derived Portfolio Created
Scope: client-strategy-high_street_pension_fund-396e-1e38-9095-26
Code: mandate-balanced
Portfolio Effective From: 2021-03-18 15:55:59.365049+00:00
Portfolio Created On: 2021-03-23 15:56:15.579047+00:00

   Parent Portfolio Details
   Scope: client-high_street_pension_fund-396e-1e2f-4280-08
   Code: mandate-balanced

Derived Portfolio Created
Scope: client-strategy-high_street_pension_fund-396e-1e38-9095-26
Code: mandate-energy
Portfolio Effective From: 2021-03-18 15:55:59.365049+00:00
Portfolio Created On: 2021-03-23 15:56:16.249243+00:00

   Parent Portfolio Details
   Scope: client-high_street_pension_fund-396e-1e2f-4280-08
   Code: mandate-energy

Derived Portfolio Created
Scope: client-strategy-high_street_pension_fund-396e-1e38-9095-26
Code: mandate-fixedincome
Portfolio Effective From: 2021-03-18 15:55:59.365049+00:00
Portfolio Created On: 2021-03-23 15:56:16.912099+00:00

   Parent Portfolio Details
   Scope: client-high_street_pension_fund-396e-1e2f-4

![Scopes](img/strategies/strategies-derived.gif)

Now that we have created our derived portfolios we can aggregate across our group of derived portfolios. Note that we can use the market data quotes from the previous aggregation in our main scope as we only had dummy prices in there.

*Run the cell below to aggregate by strategy across our client's holdings*

In [19]:
# Create the valuation request
valuation_request = models.ValuationRequest(
    recipe_id= models.ResourceId(scope=recipe_scope, code=recipe_code),
    metrics=[
        models.AggregateSpec(
            key=strategy_property_key,
            op='Value'),
        models.AggregateSpec(
            key='Holding/default/Cost',
            op='Sum')],
    filters=[
        models.PropertyFilter(
            left=strategy_property_key,
            operator='NotEquals',
            right='<Not Classified>',
            right_operand_type='Absolute'),
        models.PropertyFilter(
            left="Instrument/default/Name",
            operator='NotEquals',
            right='GBP',
            right_operand_type='Absolute'),
        models.PropertyFilter(
            left="Instrument/default/Name",
            operator='NotEquals',
            right='USD',
            right_operand_type='Absolute')
    ], 
    group_by=[strategy_property_key],
    portfolio_entity_ids=[models.PortfolioEntityId(scope=client_strategy_scope_code,
                                                   code=client_group_code,
                                                   portfolio_entity_type="GroupPortfolio")],
    valuation_schedule=models.ValuationSchedule(effective_at=datetime.now(pytz.UTC).isoformat())
)

# Perform a valuation
valuation = api_factory.build(lusid.api.AggregationApi).get_valuation(
    valuation_request=valuation_request)

prettyprint.aggregation_response_strategy(valuation, strategy_property_key)

Strategy: <Not Classified>
Strategy Cost: 445851213.71

Strategy: incomeRequirements
Strategy Cost: 0.0

Strategy: fundamentalAnalysis
Strategy Cost: 0.0

Strategy: quantitativeSignal
Strategy Cost: 0.0

Strategy: newsSentiment
Strategy Cost: 0.0

Strategy: internationalExposure
Strategy Cost: 0.0



We can now see how much we have invested into each of the different investment strategies. Our final state looks like the below.

![Scopes](img/strategies/strategies-strategyaggregation.gif)